In [1]:
# installing catboost
# Catboost == 0.22 was the version of catboost at the start of this competition
!pip install catboost==0.22 --quiet

You should consider upgrading via the '/opt/conda/bin/python3.7 -m pip install --upgrade pip' command.


In [3]:
# Importing libraries
import pandas as pd
import numpy as np
import warnings
import joblib

from tqdm import tqdm, tqdm_notebook
from functools import reduce
from time import time

from catboost import CatBoostRegressor, CatBoostClassifier
from sklearn.utils import shuffle

pd.set_option('display.max_rows', 1000)           
warnings.filterwarnings('ignore')

In [4]:
# Loading data
train = pd.read_csv('./input/Train.csv')
test = pd.read_csv('./input/Test.csv')

In [5]:
data=pd.concat([train,test]).reset_index(drop=True)
df = data[['ID', 'location', 'target']]
for c in tqdm_notebook(['temp', 'precip', 'rel_humidity', 'wind_dir','wind_spd', 'atmos_press']):
    tmp = data[c].str.split(',', expand=True)
    tmp.columns = [c + '_' + str(x) for x in tmp.columns]
    df = pd.concat([df, tmp], axis=1)

In [6]:
obj_cols = [c for c in df.select_dtypes('object').columns if c not in ['ID', 'location']]
for c in tqdm_notebook(obj_cols):
    df[c] = pd.to_numeric(df[c], errors='coerce')

In [7]:
# Feature interaction functions
# There are 4 types of interactions: product interactions, division interactions, sum interactions and divide interactions

def add_prod_interacts(df, inter_cols):   
    def apply_interacts(x, inter_cols):
        cols = [x + '_prod_' + c for c in inter_cols[inter_cols.index(x)+1:]]
        interacts_df[cols] = pd.concat([df[x] * df[c] for c in inter_cols[inter_cols.index(x)+1:]], axis=1)
    
    interacts_df = pd.DataFrame()
    _ = df[inter_cols[:-1]].apply(lambda x: apply_interacts(x.name, inter_cols))
    df = pd.concat([df, interacts_df], axis=1)
    return df


def add_div_interacts(df, inter_cols):   
    def apply_interacts(x, inter_cols):
        cols = [x + '_div_' + c for c in inter_cols[inter_cols.index(x)+1:]]
        interacts_df[cols] = pd.concat([df[x] / df[c] for c in inter_cols[inter_cols.index(x)+1:]], axis=1)
    
    interacts_df = pd.DataFrame()
    _ = df[inter_cols[:-1]].apply(lambda x: apply_interacts(x.name, inter_cols))
    df = pd.concat([df, interacts_df], axis=1)

    return df

def add_sum_interacts(df, inter_cols):   
    def apply_interacts(x, inter_cols):
        cols = [x + '_sum_' + c for c in inter_cols[inter_cols.index(x)+1:]]
        interacts_df[cols] = pd.concat([df[x] + df[c] for c in inter_cols[inter_cols.index(x)+1:]], axis=1)
    
    interacts_df = pd.DataFrame()
    _ = df[inter_cols[:-1]].apply(lambda x: apply_interacts(x.name, inter_cols))
    df = pd.concat([df, interacts_df], axis=1)

    return df

def add_diff_interacts(df, inter_cols):   
    def apply_interacts(x, inter_cols):
        cols = [x + '_diff_' + c for c in inter_cols[inter_cols.index(x)+1:]]
        interacts_df[cols] = pd.concat([df[x] - df[c] for c in inter_cols[inter_cols.index(x)+1:]], axis=1)
    
    interacts_df = pd.DataFrame()
    _ = df[inter_cols[:-1]].apply(lambda x: apply_interacts(x.name, inter_cols))
    df = pd.concat([df, interacts_df], axis=1)

    return df

In [8]:
concerned_cols = df.columns[3:729]

In [9]:
concerned_cols

Index(['temp_0', 'temp_1', 'temp_2', 'temp_3', 'temp_4', 'temp_5', 'temp_6',
       'temp_7', 'temp_8', 'temp_9',
       ...
       'atmos_press_111', 'atmos_press_112', 'atmos_press_113',
       'atmos_press_114', 'atmos_press_115', 'atmos_press_116',
       'atmos_press_117', 'atmos_press_118', 'atmos_press_119',
       'atmos_press_120'],
      dtype='object', length=726)

In [10]:
orig_cols_dict = {}
added_cols_mean = []
added_cols_max = []
added_cols_min = []
added_cols_range = []

weather_cols = ['temp', 'precip', 'rel_humidity', 'wind_dir','wind_spd', 'atmos_press']
for w in tqdm_notebook(weather_cols):
    selected_cols = [c for c in df.columns if w in c]
    orig_cols_dict[w] = pd.Series(selected_cols)
    df[f'mean_{w}'] = df[selected_cols].mean(axis=1)
    added_cols_mean.append(f'mean_{w}')
    df[f'max_{w}'] = df[selected_cols].max(axis=1)
    added_cols_min.append(f'max_{w}')
    df[f'min_{w}'] = df[selected_cols].min(axis=1)
    added_cols_max.append(f'min_{w}')
    df[f'range_{w}'] = df[f'max_{w}'] - df[f'min_{w}']
    df[f'std_{w}'] = df[selected_cols].std(axis=1)
    df[f'median_{w}'] = df[selected_cols].median(axis=1)
    df[f'skew_{w}'] = df[selected_cols].skew(axis=1)
    df[f'kurt_{w}'] = df[selected_cols].kurt(axis=1)
for num_cols in tqdm_notebook([added_cols_mean, added_cols_max, added_cols_min]):
    df = add_prod_interacts(df, num_cols)
    df = add_div_interacts(df, num_cols)
    df = add_sum_interacts(df, num_cols)
    df = add_diff_interacts(df, num_cols)

In [11]:
weather_cols = ['temp', 'precip', 'rel_humidity', 'wind_dir','wind_spd', 'atmos_press']
for w in tqdm_notebook(weather_cols):
    tmp_df = pd.DataFrame()
    tmp_df['weather_col_orig'] = orig_cols_dict[w]
    tmp_df['hours_since_start'] = tmp_df['weather_col_orig'].apply(lambda x: x.split('_')[-1])
    tmp_df['hours_since_start'] = tmp_df['hours_since_start'].astype('int')
    tmp_df['hour_of_day'] = tmp_df['hours_since_start']%24
    for hour in range(0, 24):
        selected_cols = tmp_df[tmp_df['hour_of_day'] == hour]['weather_col_orig'].tolist()
        df[f'{w}_hour_{hour}_mean'] = df[selected_cols].mean(axis=1)
        df[f'{w}_hour_{hour}_min'] = df[selected_cols].min(axis=1)
        df[f'{w}_hour_{hour}_max'] = df[selected_cols].max(axis=1)
        df[f'{w}_hour_{hour}_range'] = df[f'{w}_hour_{hour}_max'] - df[f'{w}_hour_{hour}_min']
        df[f'{w}_hour_{hour}_skew'] = df[selected_cols].skew()
        df[f'{w}_hour_{hour}_kurt'] = df[selected_cols].kurt()
        if hour - 3 > 0 and hour % 3 == 0:
            df[f'{w}_hour_{hour}_prev_hour_mean_diff'] = df[selected_cols].mean(axis=1)-df[f'{w}_hour_{hour-3}_mean']
        if hour - 5 > 0 and hour % 3 == 0:
            df[f'{w}_hour_{hour}_prev_hour_mean_diff_5'] = df[selected_cols].mean(axis=1)-df[f'{w}_hour_{hour-5}_mean']
        if hour - 6 > 0 and hour % 6 == 0:
            df[f'{w}_hour_{hour}_prev_hour_max_diff_6'] = df[selected_cols].max(axis=1)-df[f'{w}_hour_{hour-6}_max']
        if hour - 4 > 0 and hour % 2 == 0:
            df[f'{w}_hour_{hour}_prev_hour_max_diff_4'] = df[selected_cols].max(axis=1)-df[f'{w}_hour_{hour-4}_max']

In [12]:

weather_cols = ['temp', 'precip', 'rel_humidity', 'wind_dir','wind_spd', 'atmos_press']
for w in tqdm_notebook(weather_cols):
    tmp_df = pd.DataFrame()
    tmp_df['weather_col_orig'] = orig_cols_dict[w]
    tmp_df['hours_since_start'] = tmp_df['weather_col_orig'].apply(lambda x: x.split('_')[-1])
    tmp_df['hours_since_start'] = tmp_df['hours_since_start'].astype('int')
    tmp_df['hour_of_day'] = tmp_df['hours_since_start']%6
    for hour in range(0, 6):
        selected_cols = tmp_df[tmp_df['hour_of_day'] == hour]['weather_col_orig'].tolist()
        df[f'{w}6_hour_{hour}_mean'] = df[selected_cols].mean(axis=1)
        df[f'{w}6_hour_{hour}_min'] = df[selected_cols].min(axis=1)
        df[f'{w}6_hour_{hour}_max'] = df[selected_cols].max(axis=1)
        df[f'{w}6_hour_{hour}_range'] = df[f'{w}_hour_{hour}_max'] - df[f'{w}_hour_{hour}_min']
        if hour - 3 > 0 and hour % 3 == 0:
            df[f'{w}6_hour_{hour}_prev_{hour}_mean_diff'] = df[f'{w}6_hour_{hour}_mean'] - df[f'{w}6_hour_{hour-3}_mean']

In [13]:
for w in tqdm_notebook(weather_cols):
    tmp_df = pd.DataFrame()
    tmp_df['weather_col_orig'] = orig_cols_dict[w]
    tmp_df['hours_since_start'] = tmp_df['weather_col_orig'].apply(lambda x: x.split('_')[-1])
    tmp_df['hours_since_start'] = tmp_df['hours_since_start'].astype('int')
    tmp_df['hour_of_day'] = tmp_df['hours_since_start']%3
    for hour in tqdm_notebook(tmp_df['hour_of_day'].unique()):
        selected_cols = tmp_df[tmp_df['hour_of_day'] == hour]['weather_col_orig'].tolist()
        df[f'{w}3_hour_{hour}_mean'] = df[selected_cols].mean(axis=1)
        if hour > 3:
            df[f'{w}3_hour_diff_{hour}_{hour-3}'] = df[f'{w}3_hour_{hour}_mean'] - df[f'{w}3_hour_{hour-3}_mean']

In [14]:
for w in tqdm_notebook(weather_cols):
    tmp_df = pd.DataFrame()
    tmp_df['weather_col_orig'] = orig_cols_dict[w]
    tmp_df['hours_since_start'] = tmp_df['weather_col_orig'].apply(lambda x: x.split('_')[-1])
    tmp_df['hours_since_start'] = tmp_df['hours_since_start'].astype('int')
    tmp_df['hour_of_day'] = tmp_df['hours_since_start']//8
    for hour in tqdm_notebook(tmp_df['hour_of_day'].unique()):
        selected_cols = tmp_df[tmp_df['hour_of_day'] == hour]['weather_col_orig'].tolist()
        df[f'{w}_cont_hours_8_{hour}_mean'] = df[selected_cols].mean(axis=1)
        df[f'{w}_cont_hours_8_{hour}_min'] = df[selected_cols].min(axis=1)
        df[f'{w}_cont_hours_8_{hour}_max'] = df[selected_cols].max(axis=1)
        df[f'{w}_cont_hours_8_{hour}_range'] = df[f'{w}_cont_hours_8_{hour}_max'] - df[f'{w}_cont_hours_8_{hour}_min']
        if hour > 0:
            df[f'{w}_cont_hours_8_hour_diff_{hour}'] = df[f'{w}_cont_hours_8_{hour}_mean'] - df[f'{w}_cont_hours_8_{hour-1}_mean']
        if hour > 2:
            df[f'{w}_cont_hours_8_hour_diff_{hour}'] = df[f'{w}_cont_hours_8_{hour}_mean'] - df[f'{w}_cont_hours_8_{hour-3}_mean']
        if hour > 4:
            df[f'{w}_cont_hours_8_hour_diff_{hour}'] = df[f'{w}_cont_hours_8_{hour}_mean'] - df[f'{w}_cont_hours_8_{hour-5}_mean']

In [15]:
for w in tqdm_notebook(weather_cols):
    tmp_df = pd.DataFrame()
    tmp_df['weather_col_orig'] = orig_cols_dict[w]
    tmp_df['hours_since_start'] = tmp_df['weather_col_orig'].apply(lambda x: x.split('_')[-1])
    tmp_df['hours_since_start'] = tmp_df['hours_since_start'].astype('int')
    tmp_df['hour_of_day'] = tmp_df['hours_since_start']//24
    for hour in tqdm_notebook(tmp_df['hour_of_day'].unique()):
        selected_cols = tmp_df[tmp_df['hour_of_day'] == hour]['weather_col_orig'].tolist()
        df[f'{w}_cont_hours_24_{hour}_mean'] = df[selected_cols].mean(axis=1)
        df[f'{w}_cont_hours_24_{hour}_max'] = df[selected_cols].max(axis=1)
        df[f'{w}_cont_hours_24_{hour}_min'] = df[selected_cols].min(axis=1)
        if hour > 0:
            df[f'{w}_cont_hours_24_hour_diff_1_{hour}'] = df[f'{w}_cont_hours_24_{hour}_mean'] / df[f'{w}_cont_hours_24_{hour-1}_mean']
        if hour > 1:
            df[f'{w}_cont_hours_24_hour_diff_2_{hour}'] = df[f'{w}_cont_hours_24_{hour}_mean'] / df[f'{w}_cont_hours_24_{hour-2}_mean']
        if hour > 2:
            df[f'{w}_cont_hours_24_hour_diff_3_{hour}'] = df[f'{w}_cont_hours_24_{hour}_mean'] / df[f'{w}_cont_hours_24_{hour-3}_mean']
        if hour > 3:
            df[f'{w}_cont_hours_24_hour_diff_4_{hour}'] = df[f'{w}_cont_hours_24_{hour}_mean'] / df[f'{w}_cont_hours_24_{hour-4}_mean']
        if hour > 4:
            df[f'{w}_cont_hours_24_hour_diff_5_{hour}'] = df[f'{w}_cont_hours_24_{hour}_mean'] / df[f'{w}_cont_hours_24_{hour-5}_mean']

In [16]:
ID_COL, TARGET_COL = 'ID', 'target'
features = [c for c in df.columns if c not in [ID_COL, TARGET_COL]]
len(features)

2791

In [17]:
drop_features = ['location',
 'loc_altitude',
 'km2',
 'aspect',
 'dist_trunk',
 'dist_primary',
 'dist_secondary',
 'dist_tertiary',
 'dist_unclassified',
 'dist_residential',
 'popn',
 'hh',
 'hh_cook_charcoal',
 'hh_cook_firewood',]
other_features = [c for c in features if c not in drop_features]
df[other_features] = df[other_features].apply(lambda x: pd.qcut(x, 20, labels=False, duplicates='drop'))

In [18]:
train=df[df.target.notnull()].reset_index(drop=True)
test=df[df.target.isna()].reset_index(drop=True)

In [19]:
train['location']

0        C
1        D
2        A
3        A
4        A
        ..
15534    A
15535    E
15536    D
15537    D
15538    C
Name: location, Length: 15539, dtype: object

In [20]:
train.head()

,ID,location,target,temp_0,temp_1,temp_2,temp_3,temp_4,temp_5,temp_6,...,atmos_press_cont_hours_24_hour_diff_3_4,atmos_press_cont_hours_24_hour_diff_4_4,atmos_press_cont_hours_24_5_mean,atmos_press_cont_hours_24_5_max,atmos_press_cont_hours_24_5_min,atmos_press_cont_hours_24_hour_diff_1_5,atmos_press_cont_hours_24_hour_diff_2_5,atmos_press_cont_hours_24_hour_diff_3_5,atmos_press_cont_hours_24_hour_diff_4_5,atmos_press_cont_hours_24_hour_diff_5_5
0,ID_train_0,C,45.126304,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,1.0,1.0,1.0,10.0,NaN,NaN,NaN,NaN
1,ID_train_1,D,79.131702,10.0,9.0,7.0,7.0,7.0,5.0,3.0,...,18.0,18.0,16.0,16.0,16.0,9.0,14.0,13.0,16.0,16.0
2,ID_train_10,A,32.661304,18.0,17.0,19.0,16.0,9.0,9.0,10.0,...,0.0,0.0,8.0,8.0,8.0,16.0,11.0,12.0,6.0,3.0
3,ID_train_100,A,53.850238,11.0,13.0,14.0,15.0,15.0,14.0,13.0,...,17.0,9.0,9.0,9.0,9.0,14.0,8.0,14.0,17.0,12.0
4,ID_train_1000,A,177.418750,9.0,8.0,8.0,8.0,6.0,4.0,2.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [21]:

test.head()

,ID,location,target,temp_0,temp_1,temp_2,temp_3,temp_4,temp_5,temp_6,...,atmos_press_cont_hours_24_hour_diff_3_4,atmos_press_cont_hours_24_hour_diff_4_4,atmos_press_cont_hours_24_5_mean,atmos_press_cont_hours_24_5_max,atmos_press_cont_hours_24_5_min,atmos_press_cont_hours_24_hour_diff_1_5,atmos_press_cont_hours_24_hour_diff_2_5,atmos_press_cont_hours_24_hour_diff_3_5,atmos_press_cont_hours_24_hour_diff_4_5,atmos_press_cont_hours_24_hour_diff_5_5
0,ID_test_0,C,NaN,12.0,11.0,10.0,8.0,7.0,4.0,2.0,...,8.0,9.0,0.0,0.0,0.0,4.0,7.0,3.0,4.0,4.0
1,ID_test_1,C,NaN,2.0,4.0,7.0,11.0,13.0,14.0,15.0,...,7.0,14.0,2.0,2.0,2.0,13.0,10.0,12.0,11.0,15.0
2,ID_test_10,D,NaN,14.0,14.0,13.0,12.0,10.0,8.0,8.0,...,6.0,7.0,15.0,15.0,15.0,7.0,8.0,7.0,6.0,6.0
3,ID_test_100,C,NaN,2.0,6.0,10.0,12.0,14.0,15.0,16.0,...,9.0,5.0,2.0,2.0,2.0,18.0,13.0,16.0,16.0,14.0
4,ID_test_1000,D,NaN,0.0,1.0,4.0,8.0,12.0,14.0,16.0,...,18.0,17.0,17.0,17.0,17.0,12.0,9.0,15.0,17.0,16.0


In [22]:

# Creating a list of test ids in the order that they will be trained
testA =  test[test.location == 'A']
testB =  test[test.location == 'B']
testC =  test[test.location == 'C']
testD =  test[test.location == 'D']
testE =  test[test.location == 'E']

tA, tD, tE, tBC = testA.ID, testD.ID, testE.ID, test[(test.location == 'B') | (test.location == 'C')].ID
test_id = pd.concat([tA, tD, tE, tBC])

In [23]:
%%time
# Creating a list to hold predictions per seed
predzz = []
for i in tqdm_notebook(range(20)):
  # Creating a list to hold predictions per location
  # Training model per location per seed
  predictions = []
  for area in tqdm_notebook(['A', 'D', 'E'], leave = False):
    # Separating training data per location
    X = train[train.location == area]
    y = X.target
    X = X.drop(['ID', 'location', 'target'], axis = 1)

    # Shuffling data
    X, y = shuffle(X, y, random_state = 0)


    # Separating testing data per location
    tes = test[test.location == area]
    tes = tes.drop(['ID', 'location'], axis = 1)

    # Training the model and making predictions per seed, per location
    preds = CatBoostRegressor(verbose = False, random_seed = i).fit(X, y).predict(tes)
    predictions.extend(preds)

  X = train[(train.location == 'B') | (train.location == 'C')]
  y = X.target
  X = X.drop(['ID', 'location', 'target'], axis = 1)
  X, y = shuffle(X, y, random_state = 0)

  tes = test[(test.location == 'B') | (test.location == 'C')]
  tes = tes.drop(['ID', 'location'], axis = 1)
  preds = CatBoostRegressor(verbose = False, random_seed = i).fit(X, y).predict(tes)
  predictions.extend(preds)

  predzz.append(predictions)

# Averaging the predictions
preds_avg = np.mean(predzz, axis = 0)

# Post processing of the predictions
predz = [((((((((((x-0.85)*1.015)-0.85)*1.012)-0.75)*1.0095)-0.55)*1.0065)-0.8)*1.007) for x in preds_avg]
predzz = [((x-0.85)*1.015) for x in predz]

# Creating a submission file
sub_df = pd.DataFrame({'ID': test_id, 'target': predzz})
sub_df.to_csv('model_3.csv', index = False)


CPU times: user 5h 24min 56s, sys: 22min 21s, total: 5h 47min 18s
Wall time: 1h 36min 16s


In [24]:
sub_df.head(10)

,ID,target
11,ID_test_1007,32.080933
15,ID_test_1010,38.336829
16,ID_test_1011,54.867679
25,ID_test_102,42.914672
27,ID_test_1021,139.035126
31,ID_test_1025,82.619142
33,ID_test_1027,49.994876
34,ID_test_1028,55.727899
38,ID_test_1031,74.718339
40,ID_test_1033,53.105281
